<div style="text-align:right;">Python Threading 多线程</div>
<div style="text-align:right;">Brickea with Material from Mofan Python</div>

# Python Threading

## Add thread

In [1]:
import threading

获取已激活的线程数

In [8]:
threading.active_count()

5

查看所有线程信息

输出的结果是一个<_MainThread(...)>带多个<Thread(...)>。

In [9]:
threading.enumerate()

[<_MainThread(MainThread, started 4532633024)>,
 <Thread(Thread-2, started daemon 123145488060416)>,
 <Heartbeat(Thread-3, started daemon 123145493315584)>,
 <HistorySavingThread(IPythonHistorySavingThread, started 123145499643904)>,
 <ParentPollerUnix(Thread-1, started daemon 123145504899072)>]

查看现在正在运行的线程

In [10]:
threading.current_thread()

<_MainThread(MainThread, started 4532633024)>

添加线程，threading.Thread()接收参数target代表这个线程要完成的任务，需自行定义

In [11]:
def thread_job():
    print('This is a thread of %s' % threading.current_thread())

def main():
    thread = threading.Thread(target=thread_job,)   # 定义线程 
    thread.start()  # 让线程开始工作
    
if __name__ == '__main__':
    main()

This is a thread of <Thread(Thread-5, started 123145510154240)>


In [12]:
threading.current_thread()

<_MainThread(MainThread, started 4532633024)>

In [13]:
threading.enumerate()

[<_MainThread(MainThread, started 4532633024)>,
 <Thread(Thread-2, started daemon 123145488060416)>,
 <Heartbeat(Thread-3, started daemon 123145493315584)>,
 <HistorySavingThread(IPythonHistorySavingThread, started 123145499643904)>,
 <ParentPollerUnix(Thread-1, started daemon 123145504899072)>]

## Join 功能

### 不加 join() 的结果

我们让 T1 线程工作的耗时增加.

In [14]:
import threading
import time

def thread_job():
    print("T1 start\n")
    for i in range(10):
        time.sleep(0.1) # 任务间隔0.1s
    print("T1 finish\n")

added_thread = threading.Thread(target=thread_job, name='T1')
added_thread.start()
print("all done\n")

T1 start
all done


T1 finish



预想中输出的结果是否为：
```
T1 start
T1 finish
all done
```

### 加入 join() 的结果

线程任务还未完成便输出all done。如果要遵循顺序，可以在启动线程后对它调用join

In [17]:
def job():
    print(threading.current_thread())
    for i in range(10):
        time.sleep(0.1)
    print("end!")
    
added_thread = threading.Thread(target=job,name="T2")
added_thread.start()
added_thread.join()
print("all done\n")

<Thread(T2, started 123145510154240)>
end!
all done



```python
def T1_job():
    print("T1 start\n")
    for i in range(10):
        time.sleep(0.1)
    print("T1 finish\n")

def T2_job():
    print("T2 start\n")
    print("T2 finish\n")

thread_1 = threading.Thread(target=T1_job, name='T1')
thread_2 = threading.Thread(target=T2_job, name='T2')
thread_1.start() # 开启T1
thread_2.start() # 开启T2
print("all done\n")
```
输出的”一种”结果是：
```
T1 start
T2 start
T2 finish
all done
T1 finish
```
现在T1和T2都没有join，注意这里说”一种”是因为all done的出现完全取决于两个线程的执行速度， 完全有可能T2 finish出现在all done之后。这种杂乱的执行方式是我们不能忍受的，因此要使用join加以控制。

我们试试在T1启动后，T2启动前加上thread_1.join():
```python
thread_1.start()
thread_1.join() # notice the difference!
thread_2.start()
print("all done\n")
```
输出结果：
```
T1 start
T1 finish
T2 start
all done
T2 finish
```
可以看到，T2会等待T1结束后才开始运行。

如果我们在T2启动后放上thread_1.join()会怎么样呢？
```python
thread_1.start()
thread_2.start()
thread_1.join() # notice the difference!
print("all done\n")
```
输出结果：
```
T1 start
T2 start
T2 finish
T1 finish
all done
```
T2在T1之后启动，并且因为T2任务量小会在T1之前完成；而T1也因为加了join，all done在它完成后才显示。

你也可以添加thread_2.join()进行尝试，但为了规避不必要的麻烦，推荐如下这种1221的V型排布：
```python
thread_1.start() # start T1
thread_2.start() # start T2
thread_2.join() # join for T2
thread_1.join() # join for T1
print("all done\n")

"""
T1 start
T2 start
T2 finish
T1 finish
all done
"""
```

## 练习 - Threading

代码实现功能，将数据列表中的数据传入，使用四个线程处理，将结果保存在Queue中，线程执行完后，从Queue中获取存储的结果

Tips: 线程调用的函数不能有return， 所以用queue来存储

In [52]:
def job(l,q):
    for i in range (len(l)):
        l[i] = l[i]**2
    q.put(l)

def multithreading():
    q =Queue()
    threads = []
    data = [[1,2,3],[3,4,5],[4,4,4],[5,5,5]]
    
    for i in range(4):
        t = threading.Thread(target=job,args=(data[i],q))
        t.start()
        threads.append(t)
    for thread in threads:
        thread.join()
    for _ in range(q.qsize()):
        print(q.get())

if __name__=='__main__':
    multithreading()

[1, 4, 9]
[9, 16, 25]
[16, 16, 16]
[25, 25, 25]


## GIL - Python 多线程不一定有效

这次我们来看看为什么说 python 的多线程 threading 有时候并不是特别理想. 最主要的原因是就是, Python 的设计上, 有一个必要的环节, 就是 Global Interpreter Lock (GIL). 这个东西让 Python 还是一次性只能处理一个东西.

我从这里摘抄了一段对于 GIL 的解释.

   > 尽管Python完全支持多线程编程， 但是解释器的C语言实现部分在完全并行执行时并不是线程安全的。 实际上，解释器被一个全局解释器锁保护着，它确保任何时候都只有一个Python线程执行。 GIL最大的问题就是Python的多线程程序并不能利用多核CPU的优势 （比如一个使用了多个线程的计算密集型程序只会在一个单CPU上面运行）。

在讨论普通的GIL之前，有一点要强调的是GIL只会影响到那些严重依赖CPU的程序（比如计算型的）。 如果你的程序大部分只会涉及到I/O，比如网络交互，那么使用多线程就很合适， 因为它们大部分时间都在等待。实际上，你完全可以放心的创建几千个Python线程， 现代操作系统运行这么多线程没有任何压力，没啥可担心的。

### 测试 GIL 

我们创建一个 job, 分别用 threading 和 一般的方式执行这段程序. 并且创建一个 list 来存放我们要处理的数据. 在 Normal 的时候, 我们这个 list 扩展4倍, 在 threading 的时候, 我们建立4个线程, 并对运行时间进行对比.

In [56]:
import time
import copy
def job(l,q):
    res = sum(l)
    q.put(res)
    
def multithreads(l):
    q = Queue()
    threads = []
    for i in range(4):
        temp = threading.Thread(target=job,args=(copy.copy(l),q),name='T%i'% i)
        temp.start()
        threads.append(temp)
    [thread.join() for thread in threads]
    total = 0
    for i in range(q.qsize()):
        total += q.get()
    print(total)

def normal(l):
    print(sum(l))

if __name__ == '__main__':
    l = list(range(1000000))
    s_t = time.time()
    multithreads(l)
    print(time.time() - s_t)
    
    s_t = time.time()
    normal(l*4)
    print(time.time() - s_t)

1999998000000
0.06116294860839844
1999998000000
0.061856985092163086


我们的运算结果没错, 所以程序 threading 和 Normal 运行了一样多次的运算. 但是我们发现 threading 却没有快多少, 按理来说, 我们预期会要快3-4倍, 因为有建立4个线程, 但是并没有. 这就是其中的 GIL 在作怪.

## Lock - 进程锁



不使用 Lock 的情况 ¶

In [65]:
def job1():
    global A
    for i in range(10):
        print('job1 ',A)
        A+=1

def job2():
    global A
    for i in range(10):
        print('job2 ',A)
        A+=10

In [66]:
if __name__ == '__main__':
    A = 0
    thread1 = threading.Thread(target=job1)
    thread2 = threading.Thread(target=job2)

    thread1.start()
    thread2.start()

    thread1.join()
    thread2.join()

job1  0
job1  1
job1  2
job1  3
job1  4
job2  4
job2  14job1  15
job1  16
job1  17
job1  18
job1  19

job2  30
job2  40
job2  50
job2  60
job2  70
job2  80
job2  90
job2  100


使用 Lock 的情况 
lock在不同线程使用同一共享内存时，能够确保线程之间互不影响，使用lock的方法是， 在每个线程执行运算修改共享内存之前，执行lock.acquire()将共享内存上锁， 确保当前线程执行时，内存不会被其他线程访问，执行运算完毕后，使用lock.release()将锁打开， 保证其他的线程可以使用该共享内存。

函数一和函数二加锁

In [67]:
def job1():
    global A,L
    L.acquire()
    for i in range(10):
        print('job1 ',A)
        A+=1
    L.release()

def job2():
    global A,L
    L.acquire()
    for i in range(10):
        print('job2 ',A)
        A+=10
    L.release()

In [68]:
if __name__ == '__main__':
    A = 0
    L = threading.Lock()
    thread1 = threading.Thread(target=job1)
    thread2 = threading.Thread(target=job2)

    thread1.start()
    thread2.start()

    thread1.join()
    thread2.join()

job1  0
job1  1
job1  2
job1  3
job1  4
job1  5
job1  6
job1  7
job1  8
job1  9
job2  10
job2  20
job2  30
job2  40
job2  50
job2  60
job2  70
job2  80
job2  90
job2  100
